## Setup


In [ ]:
import itertools
from joblib import Parallel, delayed
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from typing import List, Tuple
import plotly.subplots as sp
from plotly.subplots import make_subplots
from concurrent.futures import ThreadPoolExecutor
import time
from functions import create_random_schedules, calculate_objective, compute_convolutions, local_search, get_v_star, powerset, get_neighborhood, build_welch_bailey_schedule, service_time_with_no_shows, create_schedule_network, create_schedule_network_var_edges, create_schedule_network_from_lists, local_search_w_intermediates

## Graph representation of scheduling problem

::: column-page

In [ ]:
# Example Usage

# Define parameters
N = 4  # Number of patients
T = 3  # Number of time intervals
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.5  # Weight for waiting time in the objective

# Create and visualize the network
fig = create_schedule_network(N=N, T=T, s=s, d=d, q=q, w=w, echo=True)
fig.show()

:::

::: column-screen

In [ ]:
# Define parameters
N = 7  # Number of patients
T = 4  # Number of time intervals
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.6  # Weight for waiting time in the objective

# Create and visualize the network
fig = create_schedule_network(N=N, T=T, s=s, d=d, q=q, w=w)
fig.show()

:::


In [ ]:
# Example Usage

# Define parameters
N = 4  # Number of patients
T = 3  # Number of time intervals
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.5  # Weight for waiting time in the objective
echo=True

# Create and visualize the network
fig = create_schedule_network_var_edges(N=N, T=T, s=s, d=d, q=q, w=w, echo=echo)
fig.show()

::: column-screen

In [ ]:
# Define parameters
N = 7  # Number of patients
T = 4  # Number of time intervals
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.6  # Weight for waiting time in the objective
echo = False
# Create and visualize the network
fig = create_schedule_network_var_edges(N=N, T=T, s=s, d=d, q=q, w=w, echo=echo)
fig.show()

:::

## Known heuristics for appointment scheduling problems

-   @welch1952appointment:
    -   Schedule 2 patients at the beginning of the schedule, to minimize risk of idle time when one patient does not show up. The remaining patients should be scheduled at intervals equal to the mean consultation time.
-   @robinson_scheduling_2003:
    -   Job allowances follow a **‘dome’ pattern**, with more time allotted to patients in the middle of the day. @wang1993static found this same result for exponentially-distributed service times.\
    -   The **first job allowance**, $x_1$, is always much lower than the other $x_i$’s, and varies only slightly with $n$.
    -   The **final job allowance**, $x_{n-1}$, is also somewhat lower than the other $x_i$’s.
    -   The **intermediate job allowances**, $x_2, \ldots, x_{n-2}$, are all about the same.

![](images/robinson-chen-dome-shape.png)


In [ ]:
from functions import create_random_schedules, calculate_objective, compute_convolutions, local_search, get_v_star, powerset, get_neighborhood, build_welch_bailey_schedule

# Assuming the necessary functions are defined elsewhere:
# get_v_star, build_welch_bailey_schedule, compute_convolutions, local_search

# Parameters
N = range(16, 20)
T = 15
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.1  # Weight for waiting time in the objective
v_star = get_v_star(T)

# Lists to store results
x_stars = []
x_initials = []  # To store initial schedules
obj_vals = []
schedules_list, objectives_list = [], []

# Iterate over each n in N
start = time.time()
for n in N:
    print(f'Running local search for schedule with N={n}')
    x = build_welch_bailey_schedule(n, T)
    x_initials.append(x)  # Store the initial schedule
    convolutions = compute_convolutions(s, n, q)
    schedules, objectives = local_search_w_intermediates(x, d, q, convolutions, w, v_star, T)
    #x_star, obj = local_search(x, d, q, convolutions, w, v_star, T)
    obj_vals.append(objectives[-1])
    x_stars.append(schedules[-1])
    schedules_list.append(schedules)
    objectives_list.append(objectives)
end = time.time()
print("Optimized Schedules:", x_stars)
print("Objective Values:", obj_vals)
print(f"Search time: {end - start:.2f} seconds")

In [ ]:
# Number of subplots needed
num_subplots = len(x_stars)

# Create a subplot figure with one chart per subplot
fig = sp.make_subplots(
    rows=num_subplots, 
    cols=1, 
    shared_xaxes=True, 
    subplot_titles=[f'n = {n}' for n in N]
)

# Add each initial and optimized schedule to its respective subplot
for idx, (x_initial, x_star) in enumerate(zip(x_initials, x_stars)):
    # Add initial schedule as a dotted line
    fig.add_trace(
        go.Scatter(
            x=list(range(T)), 
            y=x_initial, 
            mode='lines', 
            name='Initial Schedule' if idx == 0 else None,  # Show legend only once
            line=dict(dash='dot', color='blue')
        ), 
        row=idx + 1, 
        col=1
    )
    
    # Add optimized schedule as a solid line with markers
    fig.add_trace(
        go.Scatter(
            x=list(range(T)), 
            y=x_star, 
            mode='lines+markers', 
            name='Optimized Schedule' if idx == 0 else None,  # Show legend only once
            line=dict(color='red')
        ), 
        row=idx + 1, 
        col=1
    )

# Update layout properties
fig.update_layout(
    height=600 * num_subplots,  # Adjust height based on the number of subplots
    title=f"Optimal Schedules Across Different Values of N\n(T={T}, w={w})",
    xaxis_title="Time Slot (x)",
    yaxis_title="# of Patients (y)",
    template="plotly_white",
    showlegend=False  # Enable legend to distinguish between initial and optimized schedules
)

# Set consistent y-axis ticks for each subplot
for i in range(1, num_subplots + 1):
    fig.update_yaxes(tickmode='linear', tick0=0, dtick=1, row=i, col=1)

# Optionally, adjust the legend position
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

# Show the Plotly figure
fig.show()

::: column-screen

In [ ]:
for idx, (n, schedules, objectives) in enumerate(zip(N, schedules_list, objectives_list), start=1):
    print(f'Processing N={n}, number of schedules: {len(schedules)}')
    
    # Create individual network graph
    individual_fig = create_schedule_network_from_lists(
        schedules=schedules,
        objective_values=objectives,
        echo=False
    )
    
    individual_fig.update_layout(
        autosize=False,
        width=1500,
        height=1800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )
    
    # Show the individual network graph
    individual_fig.show()

:::


In [ ]:
# Function to process a single N
def process_schedule(n, T, s, d, q, w, v_star):
    print(f'Running local search for schedule with N={n}')
    x = build_welch_bailey_schedule(n, T)
    convolutions = compute_convolutions(s, n, q)
    schedules, objectives = local_search_w_intermediates(x, d, q, convolutions, w, v_star, T)
    return {
        'n': n,
        'x_initial': x,
        'schedules': schedules,
        'objectives': objectives,
        'x_star': schedules[-1],
        'obj_val': objectives[-1],
    }

# Parameters
N = range(16, 20)
T = 15
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]  # Example service time probability distribution
d = 2  # Duration threshold
q = 0.1  # No-show probability
w = 0.9  # Weight for waiting time in the objective
v_star = get_v_star(T)

# Lists to store results
results = []

start = time.time()

# Parallelize the process_schedule function using Joblib
results = Parallel(n_jobs=-1)(delayed(process_schedule)(n, T, s, d, q, w, v_star) for n in N)

end = time.time()

# Extract results
x_initials = [result['x_initial'] for result in results]
schedules_list = [result['schedules'] for result in results]
objectives_list = [result['objectives'] for result in results]
x_stars = [result['x_star'] for result in results]
obj_vals = [result['obj_val'] for result in results]

print("Optimized Schedules:", x_stars)
print("Objective Values:", obj_vals)
print(f"Search time: {end - start:.2f} seconds")

In [ ]:
# Number of subplots needed
num_subplots = len(x_stars)

# Create a subplot figure with one chart per subplot
fig = sp.make_subplots(
    rows=num_subplots, 
    cols=1, 
    shared_xaxes=True, 
    subplot_titles=[f'n = {n}' for n in N]
)

# Add each initial and optimized schedule to its respective subplot
for idx, (x_initial, x_star) in enumerate(zip(x_initials, x_stars)):
    # Add initial schedule as a dotted line
    fig.add_trace(
        go.Scatter(
            x=list(range(T)), 
            y=x_initial, 
            mode='lines', 
            name='Initial Schedule' if idx == 0 else None,  # Show legend only once
            line=dict(dash='dot', color='blue')
        ), 
        row=idx + 1, 
        col=1
    )
    
    # Add optimized schedule as a solid line with markers
    fig.add_trace(
        go.Scatter(
            x=list(range(T)), 
            y=x_star, 
            mode='lines+markers', 
            name='Optimized Schedule' if idx == 0 else None,  # Show legend only once
            line=dict(color='red')
        ), 
        row=idx + 1, 
        col=1
    )

# Update layout properties
fig.update_layout(
    height=600 * num_subplots,  # Adjust height based on the number of subplots
    title=f"Optimal Schedules Across Different Values of N\n(T={T}, w={w})",
    xaxis_title="Time Slot (x)",
    yaxis_title="# of Patients (y)",
    template="plotly_white",
    showlegend=False  # Enable legend to distinguish between initial and optimized schedules
)

# Set consistent y-axis ticks for each subplot
for i in range(1, num_subplots + 1):
    fig.update_yaxes(tickmode='linear', tick0=0, dtick=1, row=i, col=1)

# Optionally, adjust the legend position
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

# Show the Plotly figure
fig.show()

::: column-screen

In [ ]:
for idx, (n, schedules, objectives) in enumerate(zip(N, schedules_list, objectives_list), start=1):
    print(f'Processing N={n}, number of schedules: {len(schedules)}')
    
    # Create individual network graph
    individual_fig = create_schedule_network_from_lists(
        schedules=schedules,
        objective_values=objectives,
        echo=False
    )
    
    individual_fig.update_layout(
        autosize=False,
        width=1500,
        height=1800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )
    
    # Show the individual network graph
    individual_fig.show()

:::


In [ ]:
# Function to process a single N
def process_schedule(n, T, s, d, q, w, v_star):
    print(f'Running local search for schedule with N={n}')
    x = build_welch_bailey_schedule(n, T)
    convolutions = compute_convolutions(s, n, q)
    schedules, objectives = local_search_w_intermediates(x, d, q, convolutions, w, v_star, T)
    return {
        'n': n,
        'x_initial': x,
        'schedules': schedules,
        'objectives': objectives,
        'x_star': schedules[-1],
        'obj_val': objectives[-1],
    }

# Parameters
N = range(19, 22)
T = 18
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]
d = 2
q = 0.1
w = 0.9
v_star = get_v_star(T)

# Lists to store results
results = []

start = time.time()
# Use ThreadPoolExecutor for parallelism
with ThreadPoolExecutor() as executor:
    # Map the function to the range of N
    futures = [executor.submit(process_schedule, n, T, s, d, q, w, v_star) for n in N]
    for future in futures:
        results.append(future.result())

end = time.time()

# Extract results
x_initials = [result['x_initial'] for result in results]
schedules_list = [result['schedules'] for result in results]
objectives_list = [result['objectives'] for result in results]
x_stars = [result['x_star'] for result in results]
obj_vals = [result['obj_val'] for result in results]

print("Optimized Schedules:", x_stars)
print("Objective Values:", obj_vals)
print(f"Search time: {end - start:.2f} seconds")

::: column-screen

In [ ]:
for idx, (n, schedules, objectives) in enumerate(zip(N, schedules_list, objectives_list), start=1):
    print(f'Processing N={n}, number of schedules: {len(schedules)}')
    
    # Create individual network graph
    individual_fig = create_schedule_network_from_lists(
        schedules=schedules,
        objective_values=objectives,
        echo=False
    )
    
    individual_fig.update_layout(
    autosize=False,
    width=1500,
    height=1800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
        )
    )
    
    # Show the individual network graph
    individual_fig.show()

:::


In [ ]:
# Function to process a single N
def process_schedule(n, T, s, d, q, w, v_star):
    print(f'Running local search for schedule with N={n}')
    x = build_welch_bailey_schedule(n, T)
    convolutions = compute_convolutions(s, n, q)
    schedules, objectives = local_search_w_intermediates(x, d, q, convolutions, w, v_star, T)
    return {
        'n': n,
        'x_initial': x,
        'schedules': schedules,
        'objectives': objectives,
        'x_star': schedules[-1],
        'obj_val': objectives[-1],
    }

# Parameters
N = range(21, 22)
T = 20
s = [0.3, 0.2, 0.1, 0.05, 0.15, 0.2]
d = 2
q = 0.1
w = 0.9
v_star = get_v_star(T)

# Lists to store results
results = []

start = time.time()
# Use ThreadPoolExecutor for parallelism
with ThreadPoolExecutor() as executor:
    # Map the function to the range of N
    futures = [executor.submit(process_schedule, n, T, s, d, q, w, v_star) for n in N]
    for future in futures:
        results.append(future.result())

end = time.time()

# Extract results
x_initials = [result['x_initial'] for result in results]
schedules_list = [result['schedules'] for result in results]
objectives_list = [result['objectives'] for result in results]
x_stars = [result['x_star'] for result in results]
obj_vals = [result['obj_val'] for result in results]

print("Optimized Schedules:", x_stars)
print("Objective Values:", obj_vals)
print(f"Search time: {end - start:.2f} seconds")

::: column-screen

In [ ]:
for idx, (n, schedules, objectives) in enumerate(zip(N, schedules_list, objectives_list), start=1):
    print(f'Processing N={n}, number of schedules: {len(schedules)}')
    
    # Create individual network graph
    individual_fig = create_schedule_network_from_lists(
        schedules=schedules,
        objective_values=objectives,
        echo=False
    )
    
    individual_fig.update_layout(
    autosize=False,
    width=1500,
    height=1800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
        )
    )
    
    # Show the individual network graph
    individual_fig.show()

:::